In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
np.array([0,3,-3]).clip(-1,1)

In [ ]:
class Robot:
    def __init__(self):
        self.mass = np.array([1.3, 1.4, 0.7])
        self.pos = np.array([0.0, 0.0, 0.0])
        self.vel = np.array([0.0, 0.0, 0.0])
        self.drag = np.array([-0.1, -0.2, -0.4])


    def sim(self, power, dt):
        power = power.clip(-1, 1)
        a = power / self.mass
        v = self.vel + a * dt
        vdir = np.sign(v)
        dr = self.drag * np.sign(vdir)
        v += dr * dt
        dswitch = np.sign(v) != vdir
        v[dswitch] = 0
        self.vel = v

        self.pos += v * dt

In [ ]:
def sim(robot, driver, t, n):
    dt = t / n
    traj = np.empty((n, 6))
    traj[0, :3] = robot.pos
    traj[0, 3:] = robot.vel

    for i in range(1, n):
        robot.sim(driver.get_power(robot.pos, robot.vel), dt)
        traj[i, :3] = robot.pos
        traj[i, 3:] = robot.vel

    return traj

In [ ]:
def plot_traj(traj, num_arrows):
    x, y = traj[:, 0], traj[:, 1]

    stride = traj.shape[0] // num_arrows
    s = traj[::stride, :]
    qx, qy = s[:, 0], s[:, 1]
    vu, vv = s[:, 3], s[:, 4]
    hu, hv = np.cos(s[:, 2]), np.sin(s[:, 2])

    plt.figure(figsize=(6, 6))

    plt.plot(x, y)
    plt.quiver(qx, qy, vu, vv, angles='xy', color='blue', label='velocity')
    plt.quiver(qx, qy, hu, hv, angles='xy', color='green', label='head')

    plt.legend()

    plt.show()

In [ ]:
class SimpleDriveToPoint:
    def __init__(self, tgt_pos):
        self.tgt_pos = tgt_pos

        self.p = np.array([1.0, 1.0, 1.0])


    def get_power(self, pos, vel):
        return self.p * (self.tgt_pos - pos)

In [ ]:
r = Robot()
d = SimpleDriveToPoint(np.array([3.0, 2.0, 1.0]))
traj = sim(r, d, t=30, n=300)
plot_traj(traj, 12)